In [1]:
# imports numpy, pandas, matplotlib, scikit-learn, tensorflow needed
import numpy as np 
import pandas as pd
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
# import

2023-03-20 13:36:34.476298: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-20 13:36:34.876712: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-20 13:36:48.653685: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-20 13:36:48.655595: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [2]:
# num_subj = 2

# num_games = [4]
def FunctionGetSubData(n_subj, n_game): 
    n = 1
    while n <= n_subj:
        if n < 10: 
            emotions = "GAMEEMO/(S0{})/Preprocessed EEG Data/.csv format/S0{}G{}AllChannels.csv".format(n, n, n_game)
        else:
            emotions = "GAMEEMO/(S{})/Preprocessed EEG Data/.csv format/S{}G{}AllChannels.csv".format(n, n, n_game)
        data = pd.read_csv(emotions)
        data = data.drop(columns=["Unnamed: 14"])
        data["subject_num"] = n
        
        if n == 1:
            emo_df = data.iloc[15300:30600]
        else:
            # third minute of each subject's data
            emo_df2 = data.iloc[15300:30600]
            emo_df = pd.concat([emo_df, emo_df2], axis=0)
        # emotions1= emotions1.reset_index(drop=True)
        n = n+1
    
    # emotions["pred_valence"] = 1
    # emotions["pred_arousal"] = 1
    return (emo_df)
    # print(emotions_funny.info())


In [3]:
def FunctionGetGameData(n_subj, n_games):
    for game in n_games:
        # # 1 = bored (0,0); 2 = calm (1,0); 3 = horror (0,1); 4 = funny (1,1)
        if game == 1:
            emo_bored = FunctionGetSubData(n_subj, game)
            emo_bored["pred_valence"] = 0
            emo_bored["pred_arousal"] = 0
        elif game == 2:
            emo_calm = FunctionGetSubData(n_subj, game)
            emo_calm["pred_valence"] = 1
            emo_calm["pred_arousal"] = 0
        elif game == 3:
            emo_horror = FunctionGetSubData(n_subj, game)
            emo_horror["pred_valence"] = 0
            emo_horror["pred_arousal"] = 1
        elif game == 4:
            emo_funny = FunctionGetSubData(n_subj, game)
            emo_funny["pred_valence"] = 1
            emo_funny["pred_arousal"] = 1
    emotions_all = pd.concat([emo_bored, emo_calm, emo_horror, emo_funny], axis=0)
    return emotions_all



In [4]:
# only using first 8 subjects, all four games
emo_all = FunctionGetGameData(8, [1, 2, 3, 4])
print(emo_all)

# emo_bored = FunctionGetSubData(6, 1)
# print(emo_bored)

            AF3       AF4       F3         F4        F7       F8      FC5  \
15300  -9.39300   9.40330   7.3046   6.108800 -11.45820  22.6112   7.6865   
15301  -6.41920   7.15210  14.3379   3.934500   3.32930  13.4209  21.3253   
15302   6.25820   0.21479   6.4875   0.083213  17.22950   4.6619   5.9458   
15303  -8.21730 -13.89810  -8.0017 -14.021800   2.09570  -9.7179  -8.5109   
15304 -12.66530 -21.37930 -13.9085 -27.280500  -2.24810 -14.5578  -2.9388   
...         ...       ...      ...        ...       ...      ...      ...   
30595  -4.88140  -5.29560  -6.5941  -0.063464   0.61872  -6.8075   9.1474   
30596  -1.93740  -9.55710 -10.2963  -3.675100   2.26880  -0.3735  10.2858   
30597   0.58922  -9.46590 -10.6426  -8.275100   4.06070  -4.2077   5.3301   
30598  -2.57990  -7.21090  -7.8347  -7.055400   5.02210 -10.4630   5.7330   
30599  -1.94270  -7.74210  -5.9183  -4.703900  10.98770  -5.9783  13.1022   

            FC6       O1       O2       P7       P8        T7       T8  \
1

In [5]:
# Separate Target Variable and Predictor Variables
TargetVariable=['pred_valence', 'pred_arousal']
# Predictors=['AF3', 'AF4', 'F3', 'F4']
Predictors=['AF3', 'AF4', 'F3', 'F4', 'F7', 'F8' ,'FC5', 'FC6', 'O1', 'O2', 'P7', 'P8', 'T7', 'T8']
X=emo_all[Predictors].values
y=emo_all[TargetVariable].values

### Sandardization of data ###
from sklearn.preprocessing import StandardScaler
PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()

# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X)
TargetVarScalerFit=TargetVarScaler.fit(y)

# Generating the standardized values of X and y
X=PredictorScalerFit.transform(X)
y=TargetVarScalerFit.transform(y)

# Split the data into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Quick sanity check with the shapes of Training and testing datasets
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(342720, 14)
(342720, 2)
(146880, 14)
(146880, 2)


In [6]:
# importing the libraries
from tensorflow import keras
from keras.layers import Dense, Dropout
from keras.models import Sequential, load_model

from keras.optimizers import SGD
from keras.models import Sequential
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[10, 25, 50]
    epoch_list  =   [10, 20, 30]
    
    import pandas as pd
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'Accuracy'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber+=1
            # create ANN model
            model = Sequential()
            # Defining the first layer of the model
            model.add(Dense(units=10, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
 
            # applying dropout to layer
            model.add(Dropout(0.1))

            # Defining the Second layer of the model
            model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))

            # Defining the third layer of the model
            model.add(Dense(units=5, kernel_initializer='normal', activation='relu'))
 
            # applying dropout to layer
            model.add(Dropout(0.1))

            # The output neuron is a single fully connected node 
            # Since we will be predicting a single number
            model.add(Dense(2, kernel_initializer='normal'))
 
            # Compiling the model
            model.compile(loss='mean_squared_error', optimizer='adam')
 
            # Fitting the ANN to the Training set
            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=2)
 
            # Mean absolute percentage error
            MAPE = mean_absolute_percentage_error(y_test, model.predict(X_test))
            
            # printing the results of the current iteration
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'Accuracy:', 100-MAPE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), 100-MAPE]],
                                                                    columns=['TrialNumber', 'Parameters', 'Accuracy'] ))
    return(SearchResultsData)

In [ ]:
# Calling the function
ResultsData =FunctionFindBestParams(X_train, y_train, X_test, y_test)

In [ ]:
# plot results
%matplotlib inline
ResultsData.plot(x='Parameters', y='Accuracy', figsize=(15,4), kind='line')
# 25, 50

In [7]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)


def functionBestModel(batch, epoch):
    model = Sequential()
    # Defining the first layer of the model
    model.add(Dense(units=10, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))

    # applying dropout to layer
    model.add(Dropout(0.1))

    # Defining the Second layer of the model
    model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))

    # Defining the third layer of the model
    model.add(Dense(units=5, kernel_initializer='normal', activation='relu'))

    # applying dropout to layer
    model.add(Dropout(0.1))

    # The output neuron is a single fully connected node 
    # Since we will be predicting a single number
    model.add(Dense(2, kernel_initializer='normal'))

    # Compiling the model
    model.compile(loss='mean_squared_error', optimizer='adam')

    # Fitting the ANN to the Training set
    model.fit(X_train, y_train ,batch_size = batch, epochs = epoch, callbacks=[cp_callback], verbose=2)

    return model

In [8]:
model = functionBestModel(25, 50)


2023-03-20 13:38:39.887463: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-20 13:38:39.958146: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2023-03-20 13:38:39.958183: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-03-20 13:38:39.959654: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neur

Epoch 1/50

Epoch 1: saving model to training_1/cp.ckpt
13709/13709 - 11s - loss: 0.9651 - 11s/epoch - 815us/step
Epoch 2/50

Epoch 2: saving model to training_1/cp.ckpt
13709/13709 - 12s - loss: 0.9542 - 12s/epoch - 843us/step
Epoch 3/50

Epoch 3: saving model to training_1/cp.ckpt
13709/13709 - 10s - loss: 0.9496 - 10s/epoch - 763us/step
Epoch 4/50

Epoch 4: saving model to training_1/cp.ckpt
13709/13709 - 10s - loss: 0.9467 - 10s/epoch - 760us/step
Epoch 5/50

Epoch 5: saving model to training_1/cp.ckpt
13709/13709 - 12s - loss: 0.9452 - 12s/epoch - 883us/step
Epoch 6/50

Epoch 6: saving model to training_1/cp.ckpt
13709/13709 - 11s - loss: 0.9444 - 11s/epoch - 766us/step
Epoch 7/50

Epoch 7: saving model to training_1/cp.ckpt
13709/13709 - 10s - loss: 0.9440 - 10s/epoch - 739us/step
Epoch 8/50

Epoch 8: saving model to training_1/cp.ckpt
13709/13709 - 10s - loss: 0.9437 - 10s/epoch - 713us/step
Epoch 9/50

Epoch 9: saving model to training_1/cp.ckpt
13709/13709 - 10s - loss: 0.9428

In [9]:
# Generating Predictions on testing data
Predictions=model.predict(X_test)
 
# Scaling the predicted Price data back to original price scale
Predictions=TargetVarScalerFit.inverse_transform(Predictions)
 
# Scaling the y_test Price data back to original price scale
y_test_orig=TargetVarScalerFit.inverse_transform(y_test)
 
# Scaling the test data back to original scale
Test_Data=PredictorScalerFit.inverse_transform(X_test)

4590/4590 [==============================] - 3s 609us/step


In [10]:
TestingData=pd.DataFrame(data=Test_Data, columns=Predictors)
TestingData[['Valence', 'Arousal']]=y_test_orig
TestingData[['PredictedValence', 'PredictedArousal']]=Predictions
TestingData.head()

,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,O2,P7,P8,T7,T8,Valence,Arousal,PredictedValence,PredictedArousal
0,-3.0482,7.8587,-5.9801,-4.13170,-4.61150,37.3873,-4.6919,-2.0376,-9.8944,-10.7469,14.56240,-12.5265,2.7649,5.1429,1.0,0.0,0.503212,0.502107
1,-8.3637,2.5503,-2.8147,-0.53312,0.23146,-0.7941,-3.5293,3.8227,9.3997,12.1075,11.06240,8.3727,8.6241,2.1006,1.0,1.0,0.584945,0.638189
2,12.0151,-2.2099,-10.2183,-3.61710,2.08490,-33.7221,8.3567,-42.0455,-24.2231,-6.9765,23.46250,-3.0187,10.4903,-1.8339,1.0,0.0,0.495155,0.335442
3,-6.6439,-1.4744,-8.8610,3.48690,-0.74980,-10.5614,-2.2874,6.5161,1.4673,5.9673,-0.33357,-1.5761,-0.6879,-6.6033,0.0,1.0,0.501175,0.489526
4,10.9313,5.6357,-5.4788,-8.48180,11.53590,17.4337,-1.5928,-4.8819,-2.0763,-40.0687,7.07360,-9.1306,-3.8230,-35.1435,1.0,0.0,0.667146,0.693877


In [11]:
APE_val = 100*(abs((TestingData['Valence']+1)-(TestingData['PredictedValence']+1))/(TestingData['Valence']+1))
TestingData['APE_val'] = APE_val

APE_aro = 100*(abs((TestingData['Arousal']+1)-(TestingData['PredictedArousal']+1))/(TestingData['Arousal']+1))
TestingData['APE_aro'] = APE_aro

print('Accuracy of model in predicting valence is: ', 100-np.mean(APE_val))
print('Accuracy of model in predicting arousal is: ', 100-np.mean(APE_aro))

TestingData.head()

Accuracy of model in predicting valence is:  63.44309565450891
Accuracy of model in predicting arousal is:  66.04787673730168


,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,O2,P7,P8,T7,T8,Valence,Arousal,PredictedValence,PredictedArousal,APE_val,APE_aro
0,-3.0482,7.8587,-5.9801,-4.13170,-4.61150,37.3873,-4.6919,-2.0376,-9.8944,-10.7469,14.56240,-12.5265,2.7649,5.1429,1.0,0.0,0.503212,0.502107,24.839425,50.210738
1,-8.3637,2.5503,-2.8147,-0.53312,0.23146,-0.7941,-3.5293,3.8227,9.3997,12.1075,11.06240,8.3727,8.6241,2.1006,1.0,1.0,0.584945,0.638189,20.752758,18.090546
2,12.0151,-2.2099,-10.2183,-3.61710,2.08490,-33.7221,8.3567,-42.0455,-24.2231,-6.9765,23.46250,-3.0187,10.4903,-1.8339,1.0,0.0,0.495155,0.335442,25.242269,33.544183
3,-6.6439,-1.4744,-8.8610,3.48690,-0.74980,-10.5614,-2.2874,6.5161,1.4673,5.9673,-0.33357,-1.5761,-0.6879,-6.6033,0.0,1.0,0.501175,0.489526,50.117540,25.523686
4,10.9313,5.6357,-5.4788,-8.48180,11.53590,17.4337,-1.5928,-4.8819,-2.0763,-40.0687,7.07360,-9.1306,-3.8230,-35.1435,1.0,0.0,0.667146,0.693877,16.642702,69.387674


In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                150       
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 5)                 55        
                                                                 
 dropout_1 (Dropout)         (None, 5)                 0         
                                                                 
 dense_3 (Dense)             (None, 2)                 12        
                                                                 
Total params: 327
Trainable params: 327
Non-trainable pa

In [ ]:
# Loads the weights
# model.load_weights(checkpoint_path)

# # Re-evaluate the model
# loss, acc = model.evaluate(X_test, y_test, verbose=2)
# print("Restored model, accuracy: {:5.2f}%".format(100 * acc))



In [21]:
# 1 = bored (0,0); 2 = calm (1,0); 3 = horror (0,1); 4 = funny (1,1)
# def functionFindEmo(data):

pred_data = TestingData[['Valence', 'Arousal', 'PredictedValence', 'PredictedArousal',]].copy()

pred_data.loc[(pred_data['PredictedValence'] > 0.5) & (pred_data['PredictedArousal'] > 0.5), 'PredEmotionClass'] = 4
pred_data.loc[(pred_data['PredictedValence'] < 0.5) & (pred_data['PredictedArousal'] > 0.5), 'PredEmotionClass'] = 3
pred_data.loc[(pred_data['PredictedValence'] > 0.5) & (pred_data['PredictedArousal'] < 0.5), 'PredEmotionClass'] = 2
pred_data.loc[(pred_data['PredictedValence'] < 0.5) & (pred_data['PredictedArousal'] < 0.5), 'PredEmotionClass'] = 1

pred_data.loc[(pred_data['Valence'] > 0.5) & (pred_data['Arousal'] > 0.5), 'RealEmotionClass'] = 4
pred_data.loc[(pred_data['Valence'] < 0.5) & (pred_data['Arousal'] > 0.5), 'RealEmotionClass'] = 3
pred_data.loc[(pred_data['Valence'] > 0.5) & (pred_data['Arousal'] < 0.5), 'RealEmotionClass'] = 2
pred_data.loc[(pred_data['Valence'] < 0.5) & (pred_data['Arousal'] < 0.5), 'RealEmotionClass'] = 1

pred_data.loc[(pred_data['PredEmotionClass'] == pred_data['RealEmotionClass']), 'Match'] = 'True'
pred_data.loc[(pred_data['PredEmotionClass'] != pred_data['RealEmotionClass']), 'Match'] = 'False'


pred_data.head()  
pred_data.Match.value_counts()

False    92966
True     53914
Name: Match, dtype: int64